In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = r"C:\Users\omi_i\Downloads\archive\PetImages"


base_model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))
base_model.trainable = False  # freeze CNN

datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_directory(
    base_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode="binary",
    shuffle=False  # important! to keep labels aligned
)

# -----------------------
# 4️⃣ Extract Features
# -----------------------
features = base_model.predict(generator, verbose=1)
features = features.reshape(features.shape[0], -1)  # flatten to 2D
labels = generator.classes

print("Feature shape from CNN:", features.shape)  # e.g. (25000, 25088)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

xgb = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    eval_metric="logloss",
    tree_method="hist"   # much faster on CPU
)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"✅ Test Accuracy: {acc*100:.2f}%")


Found 24771 images belonging to 2 classes.


C:\Users\omi_i\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


733/775 ━━━━━━━━━━━━━━━━━━━━ 1:44 2s/step

C:\Users\omi_i\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


775/775 ━━━━━━━━━━━━━━━━━━━━ 1945s 3s/step
Feature shape from CNN: (24771, 8192)


KeyboardInterrupt: 

In [3]:
np.save("features.npy", features)
np.save("labels.npy", labels)


In [4]:
features = np.load("features.npy")
labels = np.load("labels.npy")


In [5]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Train lighter XGBoost (faster)
xgb = XGBClassifier(
    n_estimators=50,   # smaller, faster
    max_depth=4,
    learning_rate=0.1,
    subsample=0.7,
    colsample_bytree=0.7,
    eval_metric="logloss",
    tree_method="hist"  # CPU-friendly
)

xgb.fit(X_train, y_train)

# Evaluate
y_pred = xgb.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"✅ XGBoost Test Accuracy: {acc*100:.2f}%")


✅ XGBoost Test Accuracy: 86.74%


In [6]:
from sklearn.decomposition import PCA

# Reduce dimensionality
pca = PCA(n_components=256)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

xgb.fit(X_train_pca, y_train)

y_pred = xgb.predict(X_test_pca)
acc = accuracy_score(y_test, y_pred)
print(f"✅ PCA + XGBoost Accuracy: {acc*100:.2f}%")


✅ PCA + XGBoost Accuracy: 84.34%


In [7]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# If you saved your trained CNN
model = load_model("cats_dogs_cnn.h5")


In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

def grad_cam(img_path, model, last_conv_layer_name='conv2d_2'):
    # Load and preprocess image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128,128))
    img_input = np.expand_dims(img/255.0, axis=0)

    # Grad-CAM model: conv layer output + predictions
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, preds = grad_model(img_input)
        loss = preds[0][0]  # for binary classification

    # Gradients
    grads = tape.gradient(loss, conv_outputs)

    # Weighted average
    weights = tf.reduce_mean(grads, axis=(0,1,2))
    cam = np.zeros(conv_outputs.shape[1:3], dtype=np.float32)

    for i, w in enumerate(weights):
        cam += w * conv_outputs[0,:,:,i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam.numpy(), (128,128))
    cam = cam - np.min(cam)
    cam = cam / np.max(cam)

    # Overlay heatmap on image
    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), 0.6, heatmap, 0.4, 0)
    plt.figure(figsize=(6,6))
    plt.imshow(overlay)
    plt.axis('off')
    plt.show()
    pred_label = "Dog" if preds[0][0] > 0.5 else "Cat"
    print(f"Prediction: {pred_label}, confidence: {preds[0][0]:.2f}")


In [16]:
from tensorflow import keras

model_path = r"C:\Users\omi_i\Downloads\cats_dogs_cnn.keras"
model = keras.models.load_model(model_path)


In [17]:
import numpy as np
import cv2

img_size = (128,128)

# Read and preprocess one image
sample_img = cv2.imread(r"C:\Users\omi_i\Downloads\dog.jpg")
sample_img = cv2.resize(sample_img, img_size)
sample_img = sample_img / 255.0
sample_img = np.expand_dims(sample_img, axis=0)

# Call the model once
_ = model(sample_img)


In [27]:
# Save the trained CNN model
cnn_model_path = r"C:\Users\omi_i\Downloads\cats_dogs_cnn.keras"  # recommended Keras format
model.save(cnn_model_path)
print(f"✅ CNN model saved at {cnn_model_path}")


✅ CNN model saved at C:\Users\omi_i\Downloads\cats_dogs_cnn.keras


In [30]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
cnn_model = load_model(cnn_model_path)
def test_cnn_image(img_path, model, img_size=(128,128)):
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    
    pred = model.predict(img)[0][0]
    label = "Dog" if pred > 0.5 else "Cat"
    confidence = pred if pred > 0.5 else 1 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2f})")

# Example test
test_cnn_image(r"C:\Users\omi_i\Downloads\dog.jpg", cnn_model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
Prediction: Dog (confidence: 0.99)


In [41]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import joblib
# Pretrained VGG16 (no top FC layers)
cnn_model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))

# Flatten the CNN output to 1D feature vector
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Input

inp = cnn_model.input
out = Flatten()(cnn_model.output)
feature_extractor = Model(inputs=inp, outputs=out)
def extract_features(img_path, model):
    img = load_img(img_path, target_size=(128,128))
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)
    features = model.predict(arr)
    return features.flatten()

# Dataset paths
base_dir = r"C:\Users\omi_i\Downloads\archive\PetImages"
cats_dir = os.path.join(base_dir, "Cat")
dogs_dir = os.path.join(base_dir, "Dog")

X, y = [], []

for fname in os.listdir(cats_dir)[:1000]:   # limit for speed
    try:
        fpath = os.path.join(cats_dir, fname)
        X.append(extract_features(fpath, feature_extractor))
        y.append(0)  # cat
    except:
        continue

for fname in os.listdir(dogs_dir)[:1000]:
    try:
        fpath = os.path.join(dogs_dir, fname)
        X.append(extract_features(fpath, feature_extractor))
        y.append(1)  # dog
    except:
        continue

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBClassifier(n_estimators=100, max_depth=5, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)

print("✅ XGBoost accuracy:", xgb_model.score(X_test, y_test))

# Save model
joblib.dump(xgb_model, "xgb_catdog.pkl")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 772ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 641ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\omi_i\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [08:39:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ XGBoost accuracy: 0.9475


['xgb_catdog.pkl']

In [44]:
import joblib
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten

# Load trained XGBoost model
xgb_model = joblib.load("xgb_catdog.pkl")

# Define the same feature extractor as training
cnn_model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))
feature_extractor = Model(inputs=cnn_model.input, outputs=Flatten()(cnn_model.output))

# Function to extract features
def extract_features(img_path, model):
    img = load_img(img_path, target_size=(128,128))
    arr = img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)
    features = model.predict(arr)
    return features.flatten()

# Test image path
test_img = r"C:\Users\omi_i\Downloads\hh.png"

# Extract features & predict
features = extract_features(test_img, feature_extractor).reshape(1,-1)
pred = xgb_model.predict(features)[0]

# Map label
label = "Dog" if pred == 1 else "Cat"
print("Prediction:", label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 837ms/step
Prediction: Dog
